In [ ]:
train_df = pd.read_csv(train_path, header=None, index_col=None, delimiter=',')
test_df = pd.read_csv(test_path, header=None, index_col=None, delimiter=',')

In [ ]:
train_df

In [ ]:
import pandas as pd
from sklearn.utils import shuffle

# 获取原始各类别数量
l0_train, l1_train, l2_train = [train_df[train_df[0] == i] for i in [0, 1, 2]]
l0_size, l1_size, l2_size = [df.shape[0] for df in [l0_train, l1_train, l2_train]]

# 计算复制比例
l0_l1_ratio = l0_size // l1_size
l0_l2_ratio = l0_size // l2_size

# 扩充样本数量
l1_train_repeated = l1_train.sample(l0_size, replace=True)
l2_train_repeated = l2_train.sample(l0_size, replace=True)

# 合并扩充后的数据集
train_df_balanced = pd.concat([l0_train, l1_train_repeated, l2_train_repeated])

# 随机打乱数据
train_df_balanced = shuffle(train_df_balanced)

# 更新类别数量和比例
l0_size, l1_size, l2_size = [train_df_balanced[train_df_balanced[0] == i].shape[0] for i in [0, 1, 2]]
l0_l1_ratio, l0_l2_ratio = [(l0_size // l1_size), (l0_size // l2_size)]

print("After")
print(f"l0_size: {l0_size}, l1_size: {l1_size}, l2_size: {l2_size}")
print(f"l0_l1_ratio: {l0_l1_ratio}, l0_l2_ratio: {l0_l2_ratio}")

# 重置索引
train_df_balanced.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

print("train_df size: ", train_df_balanced.shape)

In [ ]:
# fill params dict before call train_cnn
params = {"input_w": 15, "input_h": 15,
          "num_classes": 3, "batch_size": 512, "epochs": 200}

In [ ]:
def train_cnn(training_df, test_df, params):
    """Trains and evaluates CNN on the given train and test data, respectively."""

    print("Training is starting ...")
    train_images = training_df.iloc[:, 2:].to_numpy()
    train_labels = training_df.iloc[:, 0]
    train_prices = training_df.iloc[:, 1]

    test_images = test_df.iloc[:, 2:].to_numpy()
    test_labels = test_df.iloc[:, 0]
    test_prices = test_df.iloc[:, 1]

    test_labels = keras.utils.np_utils.to_categorical(
        test_labels, params["num_classes"])
    train_labels = keras.utils.np_utils.to_categorical(
        train_labels, params["num_classes"])

    train_images = train_images.reshape(
        train_images.shape[0], params["input_w"], params["input_h"], 1)
    test_images = test_images.reshape(
        test_images.shape[0], params["input_w"], params["input_h"], 1)

    # CNN model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(
        params["input_w"], params["input_h"], 1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(params["num_classes"], activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy', 'mae', 'mse'])

    # metrics.accuracy_score, metrics.recall_score, metrics.average_precision_score, metrics.confusion_matrix
    train_data_size = train_images.shape[0]
    test_data_size = test_images.shape[0]

    print("model will be trained with {} and be tested with {} sample".format(
        train_data_size, test_data_size))
    # fit the model to the training data
    print("Fitting model to the training data...")
    print("")
    model.fit(train_images, train_labels,
              batch_size=params["batch_size"], epochs=params["epochs"], verbose=1, validation_data=None)

    predictions = model.predict(
        test_images, batch_size=params["batch_size"], verbose=1)
    print(model.evaluate(test_images, test_labels,
          batch_size=params["batch_size"], verbose=1))

    print("Train conf matrix: \n", confusion_matrix(np.array(reverse_one_hot(train_labels)),
                                                  np.array(reverse_one_hot(model.predict(train_images, batch_size=params["batch_size"], verbose=1)))))

    print("Test conf matrix: \n",  confusion_matrix(np.array(reverse_one_hot(test_labels)),
                                                  np.array(reverse_one_hot(predictions))))

    return predictions, test_labels, test_prices

def reverse_one_hot(predictions):
    reversed_x = []
    for x in predictions:
        reversed_x.append(np.argmax(np.array(x)))
    return reversed_x

In [ ]:
predictions, test_labels, test_prices = train_cnn(train_df_balanced, test_df, params)
result_df = pd.DataFrame({"prediction": np.argmax(predictions, axis=1),
                          "test_label": np.argmax(test_labels, axis=1),
                         "test_price": test_prices})
result_df.to_csv("cnn_result.csv", sep=',', index=None)

In [ ]:
from models.CNN import *
cnn = CNN()
train_path = '.\\dataset\\set-0\\output_phase2_train.csv'
test_path = '.\\dataset\\set-0\\output_phase2_test.csv'
train_df, test_df = cnn.preProcess(train_path=train_path, test_path=test_path) 


In [ ]:
predictions, test_labels, test_prices = cnn.train(training_df=train_df, test_df=test_df)

In [ ]:
predictions

In [ ]:
test_labels